# Lab12-3 RNN with long sequences
> RNN

### 2018.09.30.(일)

## Intro.

이전까지는 data를 Manual data creation 하였었다.<br>
근데 만약 책을 Input 하거나, 문자열이 길때는..?<br>
이를 더 잘 생성하고자 한다.

-> 자동으로 Input 하고싶다. : Better data Creation부분 !

In [1]:
import tensorflow as tf
import numpy as np

/Users/Choi-seonyeol/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## Better data Creation
Input data를 조금 더 편하게 다룰 수는 없을까? -> Input이 아무리 길어도, 우리가 손이 많이가지 않도록 modularization 하고 싶은것.

In [37]:
tf.reset_default_graph()
sample = " if you want you"
# 우리는 지금 각 글자들을 숫자들로 바꾸고 싶은 것!
idx2char = list(set(sample)) 
# index -> char # Set을 통해 unique 하게 만들어준다.
# 숫자를 문자로 지정
char2idx =  {c: i for i, c in enumerate(idx2char)} # char -> idx 
# 문자를 숫자로 바꾸기
sample_idx = [char2idx[c] for c in sample] # char to index

### hyper parameters

우리가 지정해주어야 하는  hyper parameter 도 sample data에 따라 자동으로 정해주고 싶다

In [38]:
# hyper parameters
dic_size = len(char2idx) # RNN input size (one hot size)
# input의 space 를 정해주기.
rnn_hidden_size = len(char2idx) # RNN outputsize
# hidden layer내의 node의 개수 -> 우리는, 각 char별로 가중치를 곱해줄 것이므로, input의 space를 넣는다.
num_classes = len(char2idx) # final output size ( RNN or softmax, etc...)
# 결과가 output이 어떻게 나올지에 대한것 -> 여기서는  input과 같은 space에서 나오는게 당연
batch_size = 1 # one sample data, one batch
sequence_length = len(sample) - 1 # number of lstm unfolding ( unit # )
# 우리 sequence의 길이 ( 전체 길이에서 1을 빼주어야 한다 : Input-output의 쌍이므로..)

# 이렇게, hyper parameter도 자동화해줄 수 있다. --> sample만 바뀌면 알아서 쭉 다 바뀔 수 있도록.

In [39]:
x_data = [sample_idx[:-1]] # X data sample (0 ~ n~1) hello : hell
y_data = [sample_idx[1:]]  # Y label sample ( 1 ~ n ) hello: ello
# input data 는 sequential 에서 앞의 글자,그때의 label은 바로 그 뒤의 글자가 된다! 

X = tf.placeholder(tf.int32, [None, sequence_length]) # X data
Y = tf.placeholder(tf.int32, [None, sequence_length]) # y label

X_one_hot = tf.one_hot(X, num_classes) # one hot : -> 0 1 0 0 0 0 0 0 0 
# Tensorflow에 one_hot이라는 아름다운 함수를 만들어 놓았다...!
# num_classes는 idx2char의 길이와 같을것 ! -> 문장내에서 발견되는 유일한 문자의 개수가 label의 개수가 될테니 ! 
# 이렇게 one_hot encoding을 할 때에는, dimmension에 차이가 있기 때문에, 항상 shape에 주의를 해야한다.(num_classis)

## LSTM and Loss
여기서부터, 실행시키는 것은 이전과 크게 다를 것이 없다.

In [40]:

cell = tf.contrib.rnn.BasicLSTMCell(num_units = rnn_hidden_size, state_is_tuple = True)
# num_units -> hidden_size는 dictionary size 와 같다 !
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, X_one_hot, initial_state=initial_state, dtype=tf.float32)

weights = tf.ones([batch_size, sequence_length])
# weights를 일단 1로 초기값 만들어 주는데, 이때 shape에 주의합시다. -> batch_size : 한번 batch에서의 input data size!,sequence_length : 한번 input에서 sequence 의 쌍 
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits = outputs, targets=Y, weights=weights)
# 사실 logits으로 output를 그대로 쓰는 것은 조금 문제가 있지만, 일단 쓰자.
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)
# 일단 Adam을 쓰자 !

prediction = tf.argmax(outputs, axis =2 )

## Training and Results

In [41]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(300):
        l, _ = sess.run([loss, train],feed_dict={X: x_data, Y:y_data})
        result = sess.run(prediction, feed_dict = {X: x_data})
        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print(i, "loss : ", l, "Prediction : ", "".join(result_str))

[[0 0 0 0 0 0 0 0 8 8 1 7 7 1 1]]
[0 0 0 0 0 0 0 0 8 8 1 7 7 1 1]
0 loss :  2.339797 Prediction :  yyyyyyyy  tuutt
[[0 0 0 0 0 3 3 3 8 8 8 7 7 7 8]]
[0 0 0 0 0 3 3 3 8 8 8 7 7 7 8]
1 loss :  2.2582388 Prediction :  yyyyywww   uuu 
[[0 0 0 0 0 8 8 8 8 8 1 1 7 7 7]]
[0 0 0 0 0 8 8 8 8 8 1 1 7 7 7]
2 loss :  2.1811976 Prediction :  yyyyy     ttuuu
[[0 0 0 0 0 3 3 3 9 8 1 8 0 7 7]]
[0 0 0 0 0 3 3 3 9 8 1 8 0 7 7]
3 loss :  2.0991642 Prediction :  yyyyywwwa t yuu
[[0 0 0 0 0 8 8 8 8 8 1 0 0 5 5]]
[0 0 0 0 0 8 8 8 8 8 1 0 0 5 5]
4 loss :  2.0114987 Prediction :  yyyyy     tyyoo
[[0 0 0 0 0 8 8 8 8 8 1 8 0 5 5]]
[0 0 0 0 0 8 8 8 8 8 1 8 0 5 5]
5 loss :  1.8872346 Prediction :  yyyyy     t yoo
[[0 0 0 0 5 3 3 3 8 4 1 8 0 5 5]]
[0 0 0 0 5 3 3 3 8 4 1 8 0 5 5]
6 loss :  1.8046632 Prediction :  yyyyowww nt yoo
[[0 0 0 0 5 3 3 3 8 4 1 8 0 7 7]]
[0 0 0 0 5 3 3 3 8 4 1 8 0 7 7]
7 loss :  1.7253637 Prediction :  yyyyowww nt yuu
[[0 8 0 0 5 7 3 3 8 8 1 8 0 7 7]]
[0 8 0 0 5 7 3 3 8 8 1 8 0 7 7]
8 loss 

## Really long sentence?

이제 정말 긴 문장을 한번 학습시켜 보겠다!

In [77]:
tf.reset_default_graph()
sentence = ("if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.")

이 문장에 대해서, training data set은
![lec12_14](../../img/lec12_14.png)
와 같이, 10개(batch_size)씩 input과 각각 다음 글자를 output으로 갖는 169개의 data가 될것.<br>
168개나 되므로, 우리는 minibatch로 구현할 예정입니다.

### Making dataset
위와 같은 dataset을 자동으로 만들어 주는 코드를 작성하여 봅시다.

In [78]:
char_set = list(set(sentence))
# Sentence 내의 Unique 한 character를 뽑아낸다.
char_dic = {w: i for i, w in enumerate(char_set)} # char -> idx 
# 각 character를 indexing 하기!
seq_length = 10
# 한번의 sequence 크기를 몇으로 할 지 지정

dataX = []
dataY = []
# 0번째부터, 마지막 10개의 숫자까지 돌린다 !
for i in range(0, len(sentence) - seq_length):
    x_str = sentence[i: i + seq_length]
    y_str = sentence[i + 1: i + seq_length + 1]
    print(i, x_str, "->", y_str)
    
    x = [char_dic[c] for c in x_str] # x str to index  : 즉, x_str내의 값을 index로 바꾸어 저장
    y = [char_dic[c] for c in y_str] # y str to index
    
    dataX.append(x)
    dataY.append(y)
    

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

### RNN parameters
이런 hyper parameter 까지 다 자동으로 만들어 줄 수 있다.<br>
내가 원한다면 seq_length를 내 마음대로 넣어주어도 된다.

In [79]:
data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
seq_length = 10 # Any orbitary number
batch_size = len(dataX)
# 이전까지는 데이터의 개수가 끽해야 몇개 안되서 그냥 했는데, 이번에는 169개나 되는 데이터이기 때문에,
# minibatch로 처리하기 위해 batch_size를 지정!

### LSTM and Loss

In [80]:
X = tf.placeholder(tf.int32, [None, seq_length]) # X data
Y = tf.placeholder(tf.int32, [None, seq_length]) # Y Label

X_one_hot = tf.one_hot(X, num_classes) # one_hot : 1 -> 0 1 0 0 0 0 0 0 0 0

# Make a lstm cell with hidden_size (each unit output vector size)
def lstm_cell():
    cell = tf.contrib.rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
    return cell

multi_cells = tf.contrib.rnn.MultiRNNCell([lstm_cell() for _ in range(2)], state_is_tuple=True)
outputs, _states = tf.nn.dynamic_rnn( multi_cells, X_one_hot, dtype = tf.float32)

# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, seq_length, num_classes])

weights = tf.ones([batch_size, seq_length])

sequence_loss = tf.contrib.seq2seq.sequence_loss(logits = outputs, targets = Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

### Training and Results

In [87]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(500):
        _, l, results = sess.run(
            [train, loss, outputs], feed_dict={X: dataX, Y: dataY})
    for j, result in enumerate(results):
        index = np.argmax(result, axis=1)
        print(i, j, ''.join([char_set[t] for t in index]), l)
    # Let's print the last char of each result to check it works
    results = sess.run(outputs, feed_dict={X: dataX})
    for j, result in enumerate(results):
        index = np.argmax(result, axis=1)
        if j is 0:  # print all for the first result to make a sentence
            print(''.join([char_set[t] for t in index]), end='')
        else:
            print(char_set[index[-1]], end='')

499 0 l you want 0.22883578
499 1  you want  0.22883578
499 2 tou want t 0.22883578
499 3  u want to 0.22883578
499 4 n want to  0.22883578
499 5  want to b 0.22883578
499 6 tont to bu 0.22883578
499 7 ont to bui 0.22883578
499 8 nd to buil 0.22883578
499 9 d to build 0.22883578
499 10 hdo build  0.22883578
499 11 to build a 0.22883578
499 12 h luild a  0.22883578
499 13 nluild a s 0.22883578
499 14 tuild a sh 0.22883578
499 15 uild a shi 0.22883578
499 16  ld a ship 0.22883578
499 17 ld a ship, 0.22883578
499 18 e a ship,  0.22883578
499 19  anship, d 0.22883578
499 20 tnship, do 0.22883578
499 21 nship, don 0.22883578
499 22 thip, don' 0.22883578
499 23  ip, don't 0.22883578
499 24 ep, don't  0.22883578
499 25 l, don't d 0.22883578
499 26 , don't dr 0.22883578
499 27  bon't dru 0.22883578
499 28 ton't drum 0.22883578
499 29  n't drum  0.22883578
499 30 n't drum u 0.22883578
499 31 dt drum up 0.22883578
499 32 t drum up  0.22883578
499 33 hdrum up p 0.22883578
499 34 toum up pe 0.2288